In [4]:
import pandas as pd

# === RUTAS ===
ruta_tabla_proyeccion = r"20251003/ADP_DTM_FACT.Proyeccion.csv"
ruta_tabla_items = r"20251003/ADP_DTM_DIM.Items.csv"
ruta_tabla_proyectos = r"20251003/ADP_DTM_DIM.Proyecto.csv"
ruta_tabla_capitulos = r"20251003/ADP_DTM_DIM.CapituloPresupuesto.csv"
ruta_tabla_insumos = r"20251003/ADP_DTM_DIM.Insumo.csv"

In [5]:
# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_tabla_proyeccion)
tabla_items = pd.read_csv(ruta_tabla_items, low_memory=False)
tabla_proyectos = pd.read_csv(ruta_tabla_proyectos)
tabla_capitulos = pd.read_csv(ruta_tabla_capitulos)
tabla_insumos = pd.read_csv(ruta_tabla_insumos)

In [6]:
tabla_base = tabla_proyeccion.copy()

# Merges previos
tabla_1 = pd.merge(tabla_base, tabla_proyectos, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Asegura unicidad en Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(
    tabla_2, tabla_items_unica,
    on="SkIdItems", how="left",
    suffixes=("", "_item")   # control de sufijos para items
)

# 🔧 FIX insumos: prefijar y garantizar clave única
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave_insumo = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(
    columns={c: f"Insumo_{c}" for c in cols_no_clave_insumo}
)

# (Opcional) Debug rápido de posibles overlaps antes del merge final
overlap_prev = set(tabla_3.columns).intersection(set(tabla_insumos_pref.columns))
print(f"[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {overlap_prev}")

# Merge final (sin sufijos porque ya prefijamos)
tabla_4 = pd.merge(
    tabla_3, tabla_insumos_pref,
    on="SkIdInsumo", how="left",
    validate="many_to_one"
)

print(f"[DEBUG] OK. Forma final: {tabla_4.shape}")

tabla_4.to_csv(
    "tabla_looker.csv",
    index=False,
    encoding="utf-8",
    sep=","                
)

[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {'SkIdInsumo'}
[DEBUG] OK. Forma final: (273450, 107)
[DEBUG] OK. Forma final: (273450, 107)


In [7]:
# === SELECCIÓN DE COLUMNAS ===
columnas_finales = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "Nombre Proyecto", "Capitulo Descripcion", "Item Descripcion", "Insumo_Insumo Descripcion", "Insumo_Agrupacion Descripcion",
    "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total", "Insumo_Valor Unitario", "Insumo_Valor Neto", "Insumo_Fecha Creacion",
    "Cantidad Item", "Macroproyecto Descripcion", "Insumo_Fecha Modificacion",
    "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalización", "SkIdFecha", "Capitulo Numero", "Cantidad_Item"
]

# Filtrar solo las columnas que existan realmente (por seguridad)
columnas_existentes = [col for col in columnas_finales if col in tabla_4.columns]
tabla_looker = tabla_4[columnas_existentes].copy()

# === EXPORTAR ===
tabla_looker.to_csv(
    "tabla_looker_final.csv",
    index=False,
    encoding="utf-8",
    sep=",")

print(f"✅ Tabla final exportada con {len(tabla_looker)} filas y {len(tabla_looker.columns)} columnas.")

✅ Tabla final exportada con 273450 filas y 22 columnas.


# Código arreglado - Diana 

In [8]:
import pandas as pd

# === RUTAS ===
ruta_tabla_proyeccion = r"20251003/ADP_DTM_FACT.Proyeccion.csv"
ruta_tabla_items = r"20251003/ADP_DTM_DIM.Items.csv"
ruta_tabla_proyectos = r"20251003/ADP_DTM_DIM.Proyecto.csv"
ruta_tabla_capitulos = r"20251003/ADP_DTM_DIM.CapituloPresupuesto.csv"
ruta_tabla_insumos = r"20251003/ADP_DTM_DIM.Insumo.csv"

# === OPCIÓN: ruta de salida ===
ruta_salida = r"20251003/Tabla_Final_ARPRO_Completa.csv"

# === 1) Cargar y preparar Proyeccion ===
proyeccion = pd.read_csv(ruta_tabla_proyeccion, low_memory=False)

columnas_proyeccion = [
    "SkIdProyecto",
    "SkIdCapitulo",
    "SkIdItems",
    "SkIdInsumo",
    "SkIdFecha",
    "SkIdFecha Real",
    "Cantidad",
    "Valor Unitario",
    "Valor Total"
]

missing = [c for c in columnas_proyeccion if c not in proyeccion.columns]
if missing:
    raise KeyError(f"Faltan columnas en Proyeccion: {missing}")

proyeccion_filtrada = proyeccion[columnas_proyeccion].copy()

proyeccion_filtrada.rename(columns={
    "Cantidad": "Cantidad_Insumo",
    "Valor Unitario": "Valor Unitario_Insumo",
    "Valor Total": "Valor Total_Item"
}, inplace=True)

# === 2) Cargar y preparar Proyecto ===
proyecto = pd.read_csv(ruta_tabla_proyectos, low_memory=False)

columnas_proyecto = [
    "SkIdProyecto",
    "Codigo Proyecto",
    "Nombre Proyecto",
    "Clase Proyecto",
    "Estado",
    "MacroProyecto Descripcion",
    "Fecha De Elaboracion",
    "Fecha De Inicio",
    "Fecha De Finalizacion"
]

missing = [c for c in columnas_proyecto if c not in proyecto.columns]
if missing:
    raise KeyError(f"Faltan columnas en Proyecto: {missing}")

proyecto_filtrado = proyecto[columnas_proyecto].copy()

proyecto_filtrado.rename(columns={
    "Estado": "Estado_Proyecto",
    "MacroProyecto Descripcion": "MacroProyecto Descripcion_Proyecto",
    "Fecha De Elaboracion": "MacroProyecto Fecha De Elaboracion_Proyecto",
    "Fecha De Inicio": "MacroProyecto Fecha De Inicio_Proyecto",
    "Fecha De Finalizacion": "MacroProyecto Fecha De Finalizacion_Proyecto"
}, inplace=True)

merge_proyeccion_proyecto = pd.merge(
    proyeccion_filtrada,
    proyecto_filtrado,
    on="SkIdProyecto",
    how="left",
    validate="m:1"
)

# === 3) Cargar Items ===
items = pd.read_csv(ruta_tabla_items, low_memory=False)

columnas_items = ["SkIdItems", "Item Descripcion"]
missing = [c for c in columnas_items if c not in items.columns]
if missing:
    raise KeyError(f"Faltan columnas en Items: {missing}")

items_filtrado = items[columnas_items].copy()

merge_items = pd.merge(
    merge_proyeccion_proyecto,
    items_filtrado,
    on="SkIdItems",
    how="left",
    validate="m:1"
)

# === 4) Cargar Insumo ===
insumo = pd.read_csv(ruta_tabla_insumos, low_memory=False)

columnas_insumo = [
    "SkIdInsumo",
    "Insumo Descripcion",
    "Fecha Creacion",
    "Fecha Modificacion"
]

missing = [c for c in columnas_insumo if c not in insumo.columns]
if missing:
    raise KeyError(f"Faltan columnas en Insumo: {missing}")

insumo_filtrado = insumo[columnas_insumo].copy()

insumo_filtrado.rename(columns={
    "Fecha Creacion": "Fecha Creacion_Insumo",
    "Fecha Modificacion": "Fecha Modificacion_Insumo"
}, inplace=True)

tabla_intermedia = pd.merge(
    merge_items,
    insumo_filtrado,
    on="SkIdInsumo",
    how="left",
    validate="m:1"
)

# === 5) Cargar CapituloPresupuesto ===
capitulo_presupuesto = pd.read_csv(ruta_tabla_capitulos, low_memory=False)

columnas_capitulo = ["Codigo Proyecto", "Capitulo Descripcion"]
missing = [c for c in columnas_capitulo if c not in capitulo_presupuesto.columns]
if missing:
    raise KeyError(f"Faltan columnas en CapituloPresupuesto: {missing}")

capitulo_filtrado = capitulo_presupuesto[columnas_capitulo].copy()

if "Codigo Proyecto" not in tabla_intermedia.columns:
    proyecto_cod = proyecto[["SkIdProyecto", "Codigo Proyecto"]].drop_duplicates()
    tabla_intermedia = pd.merge(
        tabla_intermedia,
        proyecto_cod,
        on="SkIdProyecto",
        how="left",
        validate="m:1"
    )

tabla_final = pd.merge(
    tabla_intermedia,
    capitulo_filtrado,
    on="Codigo Proyecto",
    how="left",
    validate="m:m"
)

# === 6) Rellenar valores faltantes según tus condiciones ===
tabla_final["MacroProyecto Descripcion_Proyecto"] = tabla_final["MacroProyecto Descripcion_Proyecto"].fillna("No pertenece a un Macro Proyecto")
tabla_final["Insumo Descripcion"] = tabla_final["Insumo Descripcion"].fillna("Descripcion No Disponible")
tabla_final["Fecha Modificacion_Insumo"] = tabla_final["Fecha Modificacion_Insumo"].fillna("2009-12-31")
tabla_final["Fecha Creacion_Insumo"] = tabla_final["Fecha Creacion_Insumo"].fillna("2009-12-31")

# === 7) Guardar resultado a CSV ===
tabla_final.to_csv(ruta_salida, index=False)

print("✅ Proceso completado. Archivo guardado en:", ruta_salida)

✅ Proceso completado. Archivo guardado en: 20251003/Tabla_Final_ARPRO_Completa.csv


In [10]:
import polars as pl

df = pl.read_csv(ruta_salida)

In [11]:
df.shape

(7488696, 22)

In [15]:
len(df['Nombre Proyecto'].unique())

76

In [16]:
# === EXPORTAR CSV POR PROYECTO ===
import os
import re
import unicodedata
import pandas as pd

# 1) Obtener el DataFrame fuente de forma robusta
_df = None

# a) Priorizar 'df' si ya existe (puede ser pandas o polars)
try:
    if 'df' in globals():
        _candidate = df
        # Si es Polars, convertir a pandas
        if hasattr(_candidate, 'to_pandas'):
            _candidate = _candidate.to_pandas()
        if isinstance(_candidate, pd.DataFrame):
            _df = _candidate.copy()
except Exception:
    pass

# b) Si no, usar 'tabla_final' si existe en memoria
if _df is None:
    try:
        if 'tabla_final' in globals() and isinstance(tabla_final, pd.DataFrame):
            _df = tabla_final.copy()
    except Exception:
        pass

# c) Si no, intenta leer desde 'ruta_salida' si existe la variable y el archivo
if _df is None:
    try:
        if 'ruta_salida' in globals() and isinstance(ruta_salida, str) and os.path.exists(ruta_salida):
            _df = pd.read_csv(ruta_salida, low_memory=False)
    except Exception:
        _df = None

# d) Si no, intenta con salidas previas de este notebook
if _df is None:
    for candidato in ["tabla_looker_final.csv", "tabla_looker.csv"]:
        if os.path.exists(candidato):
            _df = pd.read_csv(candidato, low_memory=False)
            break

if _df is None:
    raise FileNotFoundError(
        "No se encontró un DataFrame en memoria ('df' o 'tabla_final') ni archivos de salida ('ruta_salida', 'tabla_looker_final.csv', 'tabla_looker.csv'). "
        "Ejecuta antes las celdas que generan la tabla final."
    )

# 2) Validar columna de proyecto
col_proyecto = "Nombre Proyecto"
if col_proyecto not in _df.columns:
    # Ayuda: sugerir columnas relacionadas
    candidatos = [c for c in _df.columns if "proyecto" in c.lower() or "nombre" in c.lower()]
    raise KeyError(
        f"La columna '{col_proyecto}' no existe en los datos. Candidatos encontrados: {candidatos}"
    )

# 3) Crear carpeta de salida
carpeta_salida = "tablasProyect"
os.makedirs(carpeta_salida, exist_ok=True)

# 4) Función para sanear nombres de archivo en Windows
invalid_re = re.compile(r"[^A-Za-z0-9._-]+")

def slugify(nombre: str, max_len: int = 120) -> str:
    if not isinstance(nombre, str):
        nombre = str(nombre) if nombre is not None else ""
    # Normalizar y quitar acentos
    nombre = unicodedata.normalize("NFKD", nombre).encode("ascii", "ignore").decode("ascii")
    # Reemplazar separadores por guiones bajos
    nombre = invalid_re.sub("_", nombre).strip("._-")
    if not nombre:
        nombre = "SIN_NOMBRE"
    # Limitar longitud
    return nombre[:max_len]

# 5) Exportar un CSV por proyecto (evitando colisiones de nombre)
proyectos = (
    _df[col_proyecto]
    .fillna("SIN_NOMBRE")
    .astype(str)
    .unique()
)

usados = {}
exportados = 0

for nombre in proyectos:
    df_proj = _df[_df[col_proyecto].astype(str) == nombre]
    base = slugify(nombre)
    # Evitar colisiones si dos nombres diferentes se convierten al mismo slug
    if base in usados:
        usados[base] += 1
        fname = f"{base}_{usados[base]}.csv"
    else:
        usados[base] = 1
        fname = f"{base}.csv"

    ruta_out = os.path.join(carpeta_salida, fname)
    df_proj.to_csv(ruta_out, index=False, encoding="utf-8")
    exportados += 1

print(f"✅ Exportación completada: {exportados} archivos creados en '{carpeta_salida}'.")
print("Ejemplos de archivos:", list(usados.keys())[:5])

✅ Exportación completada: 76 archivos creados en 'tablasProyect'.
Ejemplos de archivos: ['URBAN_PLAZA', 'Parque_Engativa_-_Etapa_I', 'Edificio_Naia', 'PARQUE_ENGATIVA_ETAPA_II', 'Caminos_de_Sie_-_Urb_y_Zonas_Comunes_MZ_4']
